In [1]:
cd ~/MultiFidelity-ProcessOpt/1.\ Code

/Users/k23070952/MultiFidelity-ProcessOpt/1. Code


In [ ]:
from ShortCutDesign import ShortCutDesign 
from RigorousDesign import RigorousDesign 
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import os
import numpy as np
import torch
from scipy.stats import norm

In [3]:

# ✅ 파일 경로 설정
results_file = '../3. Data/gp_bo_data_250403.csv'
error_file = '../3. Data/gp_bo_errors_250403.csv'


# ✅ Shortcut 및 Rigorous 결과 계산 함수 
def calculate_shortcut(_init_params):
    shortcut_model = ShortCutDesign()
    shortcut_results = shortcut_model.shortcut_results(_init_params)
    return shortcut_results

def calculate_rigorous(_init_params):
    shortcut_results = calculate_shortcut(_init_params)
    if shortcut_results['CAPEX'] == 0:
        return (shortcut_results, [0, 0, 0])

    if shortcut_results['CAPEX'] is None or np.isnan(shortcut_results['CAPEX']):
        return (shortcut_results, [0, 0, 0])

    RigorousCal = RigorousDesign()
    Rigorous_results = RigorousCal.func(_init_params, shortcut_results)
    return (shortcut_results, Rigorous_results)



In [4]:
# ✅ 평가 함수 (병렬 실행용)

bounds = [
    Integer(1, 50, name="n1"),  # Extractor stages
    Real(0, 0.9999, name="Lr1"),
    Real(0, 0.9999, name="Hr1"),
    Real(0, 0.9999, name="Lr2"),
    Real(0, 0.9999, name="Hr2"),
    Real(273, 350, name="T_hex"),
    Real(0, 0.9999, name="Lr3"),
    Real(0, 0.9999, name="Hr3")
]

opt_history = []

@use_named_args(bounds)
def objective(**params):
    X = [params[key] for key in ["n1", "Lr1", "Hr1", "Lr2", "Hr2", "T_hex", "Lr3", "Hr3"]]
    try:
        # 계산 수행
        shortcut_obj, rigorous_obj = calculate_rigorous(X)

        msp = rigorous_obj[-1]
        
        # 결과 데이터 정리
        record = list(X) + [
            shortcut_obj['CAPEX'], shortcut_obj['OPEX'], shortcut_obj['AceticAcidWt'], shortcut_obj['SplitRatio'], 
            shortcut_obj['boilup_1'], shortcut_obj['N_stages_1'], shortcut_obj['feed_stage_1'], 
            shortcut_obj['boilup_2'],shortcut_obj['N_stages_2'],shortcut_obj['feed_stage_2'],
            shortcut_obj['boilup_3'],shortcut_obj['N_stages_3'],shortcut_obj['feed_stage_3'],shortcut_obj['shortcut_time'],
            rigorous_obj[0], rigorous_obj[1], rigorous_obj[2], rigorous_obj[3], msp, None
        ]

        # rigorous_result = (CAPEX, OPEX, AceticAcid_wt, time)
          # 예: CAPEX + OPEX

        # 실시간 CSV 저장
        df = pd.DataFrame([record])
        df.to_csv(results_file, mode='a', header=not os.path.exists(results_file), index=False)

        return msp

    except Exception as e:
        # 실패한 경우에는 파라미터와 에러 메시지만 저장
        record = list(X) + [None]*21 + [str(e)]  # 앞에 8개 + 21개 + error
        df = pd.DataFrame([record])
        df.to_csv(results_file, mode='a', header=not os.path.exists(results_file), index=False)
        
        return 10000

In [ ]:


# Run Bayesian Optimization with GPR
res = gp_minimize(
    func=objective,
    dimensions=bounds,
    n_calls=2000,
    n_initial_points=100,
    random_state=42,
    verbose=True
)



Iteration No: 1 started. Evaluating function at random point.
 ##### An instance of the 'BlackBox' class  has been initialised!
[40, 0.18341644638717722, 0.7796130311727423, 0.5967904729306924, 0.4457881695783059, 280.6980685179862, 0.4592029670766707, 0.333675240277908]
setting base model
setting model:  {'SplitRatio': 0.287, 'boilup_1': 4.48, 'N_stages_1': 9, 'feed_stage_1': 7, 'boilup_2': 4.48, 'N_stages_2': 9, 'feed_stage_2': 7, 'boilup_3': 4.48, 'N_stages_3': 9, 'feed_stage_3': 7}
10000.0
 ##### An instance of the 'RigorousDesign' class  has been initialised!
[40, 0.18341644638717722, 0.7796130311727423, 0.5967904729306924, 0.4457881695783059, 280.6980685179862, 0.4592029670766707, 0.333675240277908]
setting model:  {'CAPEX': 1.6629, 'OPEX': 3.4506, 'AceticAcidWt': 0.04454200357104111, 'SplitRatio': 0.01380670611439842, 'boilup_1': 0.2437120001482147, 'N_stages_1': 4, 'feed_stage_1': 3, 'boilup_2': 1.5059112289316585, 'N_stages_2': 6, 'feed_stage_2': 6, 'boilup_3': 1.9836798236929

In [ ]:


# 🔁 MC Dropout 기반 예측 함수 (평균 + 표준편차)
def mc_predict(model, x_tensor, num_samples=1000):
    model.train()  # dropout 활성화
    preds = [model(x_tensor).detach().numpy() for _ in range(num_samples)]
    preds = np.stack(preds, axis=0)
    mean = preds.mean(axis=0).squeeze()
    std = preds.std(axis=0).squeeze()
    return mean, std

# ✅ EI 함수 구현
def acquisition_EI(x, model, f_best, num_samples=1000, xi=0.01):
    x_tensor = torch.tensor(np.array(x).reshape(-1, 1), dtype=torch.float32)
    mu, sigma = mc_predict(model, x_tensor, num_samples)
    
    sigma = np.maximum(sigma, 1e-8)  # 0 나누기 방지
    z = (f_best - mu - xi) / sigma
    ei = (f_best - mu - xi) * norm.cdf(z) + sigma * norm.pdf(z)
    return -ei  # 최대화를 위한 EI → 최소화로 변환

# ✅ 다음 탐색점 찾기 (DE로 EI 최대화)
def suggest_next_point(model, f_best, bounds=[(0, 1)], num_samples=1000):
    result = differential_evolution(
        func=lambda x: acquisition_EI(x, model, f_best, num_samples),
        bounds=bounds,
        maxiter=100,
        seed=0
    )
    return result.x[0]


[{'params': {'n1': 42,
   'Lr1': 0.18341644638717722,
   'Hr1': 0.7796130311727423,
   'Lr2': 0.5967904729306924,
   'Hr2': 0.4457881695783059,
   'T_hex': 280.6980685179862,
   'Lr3': 0.4592029670766707,
   'Hr3': 0.333675240277908},
  'target': 100},
 {'params': {'n1': 16,
   'Lr1': 0.6508233841015582,
   'Hr1': 0.05640593786919756,
   'Lr2': 0.7219265723895982,
   'Hr2': 0.9384588537448488,
   'T_hex': 273.0599649697581,
   'Lr3': 0.9921123381352887,
   'Hr3': 0.6174197614767539},
  'target': 100},
 {'params': {'n1': 34,
   'Lr1': 0.007065598589195436,
   'Hr1': 0.02306011879891162,
   'Lr2': 0.5247221827923634,
   'Hr2': 0.39982098561808405,
   'T_hex': 276.5932560674484,
   'Lr3': 0.9736581432895752,
   'Hr3': 0.23274806329626127},
  'target': 100},
 {'params': {'n1': 14,
   'Lr1': 0.6183241707321541,
   'Hr1': 0.3824237450680361,
   'Lr2': 0.9831325627182077,
   'Hr2': 0.46671621695865523,
   'T_hex': 339.2154113186967,
   'Lr3': 0.6802395078339211,
   'Hr3': 0.45045420204434616}